In [11]:
%matplotlib inline

import glob
import xml.etree.ElementTree as ET

import os
import shutil
import time

from IPython.display import Image
import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

import DataLoader
from densenet_modified import densenet201

In [56]:
# def load_xml():
object_files = glob.glob("../../data/objects/train/*/*/*.xml")
object_files.sort()

objects = []

for path in object_files:
    with open(path, "r") as f:
        xml = f.read()
        xml = """<?xml version="1.0"?>
        <base>
        """ + xml + """
        </base>
        """
        
        tree = ET.fromstring(xml)
        
        path = os.path.join("../../data/images/train", tree.find("folder").text, tree.find("filename").text)
        
        objs = []
        for obj in tree.findall("objects"):
            bndbox = obj.find("bndbox")
            objdata = {
                "class": int(obj.find("class").text),
                # "polygon": obj.find("polygon"),  # ignoring polygon for now
                "bndbox": (
                    (
                        int(bndbox.find("xmin").text),
                        int(bndbox.find("xmax").text)
                    ),
                    (
                        int(bndbox.find("ymin").text),
                        int(bndbox.find("ymax").text)
                    )
                )
            }
            objs.append(objdata)
        
        data = {
            "path": path,
            "class": int(tree.find("class").text),
            "objects": objs,
        }
        objects.append(data)

In [58]:
sum(map(lambda o: len(o['objects']), objects))

46706